#Text Analytics | Assignment 1 | Additional Info

1st Approach

In [ ]:
from collections import Counter
from nltk.util import ngrams

# Unigram, Bigram, Trigram model creation

unigram_counter = Counter()
bigram_counter = Counter()
trigram_counter = Counter()

for sent in train:
    #print(sent)
    unigram_counter.update([gram for gram in ngrams(sent, 1, pad_left=True, pad_right=True,
                                                   left_pad_symbol='<s>',right_pad_symbol='<e>') ]) 
    bigram_counter.update([gram for gram in ngrams(sent, 2, pad_left=True, pad_right=True,
                                                   left_pad_symbol='<s>',right_pad_symbol='<e>') ])
    trigram_counter.update([gram for gram in ngrams(sent, 3, pad_left=True, pad_right=True,
                                                   left_pad_symbol='<s>',right_pad_symbol='<e>') ])

In [ ]:
import math

# We should set the a parameter to 1, in order to utilize the Laplace Smoothing 
alpha = 1

# Vocabulary size calculation
vocab_size = len(set(voc))

#set the initial sums equal to 0 
s_bi=0
s_tri=0

my_dict = {}

#Sum of Bigram log probabilitis
for i in list(bigram_counter.keys()):
  bigram_prob = (bigram_counter[i] + alpha) / (unigram_counter[(i[0],)] + alpha*vocab_size)
  bigram_log_prob = math.log2(bigram_prob)
  my_dict.update({i: bigram_prob})
  #print("P(the, Department) : {0:.3f} ".format(bigram_prob))
  #print(f"{i}",bigram_prob)
  s_bi+=bigram_log_prob
  #print(f"{i}",bigram_log_prob)

#Sum of Trigram log probabilitis
for i in list(trigram_counter.keys()):
  trigram_prob = (trigram_counter[i] + alpha) / (bigram_counter[(i[0],i[1],)] + alpha*vocab_size)
  trigram_log_prob = math.log2(trigram_prob)
  #print("P(the, Department) : {0:.3f} ".format(trigram_prob))
  #print(f"{i}",trigram_prob)
  s_tri+=trigram_log_prob
  #print(f"{i}",trigram_log_prob)
 
# Print the total sum of Bigram log-probabilities
print("Sum of Bigram Log-Probabilities: ",s_bi)
print("Sum of Trigram Log-Probabilities: ",s_tri)

Sum of Bigram Log-Probabilities:  -54298.86941075823
Sum of Trigram Log-Probabilities:  -133557.15935830458


In [ ]:
my_dict = {}

for i in [('<s>', 'And')]:
  bigram_prob = (bigram_counter[i] + alpha) / (unigram_counter[(i[0],)] + alpha*vocab_size)
  my_dict.update({i: bigram_prob})

In [ ]:
my_dict

{('<s>', 'And'): 2.072093023255814}

2nd Approach

In [ ]:
from collections import Counter
from nltk.util import ngrams
from pprint import pprint


unigram_counter = Counter()
bigram_counter = Counter()
trigram_counter = Counter()


# Unigram, Bigram, Trigram model creation
for sent in sentences_tokenized:

    # Update the unigram counter
    unigram_counter.update([(gram,) for gram in ["<s>"] + sent])
    
    # Update the bigram counter
    bigram_pad_sent = ["<s>"] + sent +  ['<e>']    
    bigram_counter.update([(gram1, gram2) for gram1, gram2 in zip(bigram_pad_sent, bigram_pad_sent[1:])])

    # Update the trigram counter
    trigram_pad_sent = ["<s>"]*2 + sent +  ['<e>']*2
    trigram_counter.update([(gram1, gram2, gram3) for gram1, gram2, gram3 in zip(trigram_pad_sent, trigram_pad_sent[1:], trigram_pad_sent[2:])]) 


pprint(unigram_counter.most_common(10))   
pprint(bigram_counter.most_common(10))
pprint(trigram_counter.most_common(10))

In [ ]:
my_dict = {}

for i in [('<s>','<s>', 'And')]:
  trigram_prob = (trigram_counter[i] + alpha) / (bigram_counter[(i[0],i[1])] + alpha*vocab_size)
  my_dict.update({i: trigram_prob})

In [ ]:
my_dict

{('<s>', '<s>', 'And'): 2.5930232558139537}

Levensthein Ratio

In [ ]:
#Beam search for mass input 
import Levenshtein
from collections import defaultdict

def beam_search_t_test(testList, tri_prob, vocabulary, beam_width=3):
    #Add start tokens
    t_list = copy.deepcopy(testList)
    t_list.insert(0, "<s>")
    t_list.insert(0, "<s>")

    #Clear the existing tree
    current_nodes = []

    #Tokenise sentence
    words = t_list
    current_nodes = [(words[0] + " " + words[1], 1.0, [words[0], words[1]])]

    #For every word in the sentence
    for i in range(2, len(words)):
        next_nodes = []
        #For every selected parent node

        #For every candidate node
        for node in current_nodes:
            closest_words = sorted(vocabulary, key=lambda x: Levenshtein.distance(x, words[i]))[:20]

            #For every candidate node
            for candidate in closest_words:
                #Exception for absent key
                try:
                    model_prob=tri_prob[node[0],candidate]
                except KeyError:
                    model_prob = 0
                
                #Language model and Levenshtein weights
                l1 = 0.55
                l2 = 0.45

                #Calculate probability
                prob = (l1*model_prob) + (l2*pow((Levenshtein.ratio(candidate, words[i])),2))

                #Update current path of nodes
                lineage = node[2] + [candidate]
                next_nodes.append((candidate, prob * node[1], lineage))

        #Sort by probability in descending order and save top 3
        current_nodes = sorted(next_nodes, key=lambda x: x[1], reverse=True)[:beam_width]
    return [(node[2], node[1]) for node in current_nodes]

#Show reults
print(beam_search_t_test(testing_list, tri_prob, voc))

In [ ]:
from evaluate import load
from statistics import mean

wer = load("wer")

#Create an empty list for saving the corresponding scores
bi_wer_scores=[]
for i in range(len(test)): 
  #Use the already created "bsc_caller" function and the "wer.compute" to calculate the WER score
  wer_score = wer.compute(predictions= bsc_caller(test[i]), references=test[i])
  #Store the scores to a list
  bi_wer_scores.append(wer_score)
#Calculate & print the average of the WER scores
print("Biagram model WER:", mean(bi_wer_scores))
u_error =  mean(bi_wer_scores)

Biagram model WER: 0.12615553795371387


In [ ]:
from evaluate import load
from statistics import mean

wer = load("wer")

#Create an empty list for saving the corresponding scores
bi_wer_scores=[]
for i in range(len(test)): 
  #Use the already created "bsc_caller" function and the "wer.compute" to calculate the WER score
  wer_score = wer.compute(predictions= bsc_caller(rep_test[i]), references=test[i])
  #Store the scores to a list
  bi_wer_scores.append(wer_score)
#Calculate & print the average of the WER scores
print("Biagram model WER:", mean(bi_wer_scores))
f_error = mean(bi_wer_scores)

Biagram model WER: 0.1874853821795896


In [ ]:
true_b_error = f_error - u_error
print(true_b_error)

0.06132984422587573


In [ ]:
unknown = 0 
all = 0
for i in range(len(test)):
  for j in range(len(test[i])):
    all = all +1
    if (test[i][j]=="*UNK*"):
      unknown = unknown + 1

print(unknown)
print(all)
print(unknown/all)    